In [3]:
conda install pymongo

Solving environment: done

## Package Plan ##

  environment location: /Users/administrator/opt/anaconda3

  added / updated specs:
    - pymongo


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.2                |           py37_0         2.8 MB
    pymongo-3.9.0              |   py37h0a44026_0         1.1 MB
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

  pymongo            pkgs/main/osx-64::pymongo-3.9.0-py37h0a44026_0

The following packages will be UPDATED:

  conda                                        4.8.1-py37_0 --> 4.8.2-py37_0



conda-4.8.2          | 2.8 MB    | ##################################### | 100% 
pymongo-3.9.0        | 1.1 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transactio

In [2]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
import requests

client = MongoClient('localhost', 27017)
yelp = client['yelp']
restaurants = yelp['restaurants']
processed = yelp['processed']
users = yelp['users']

In [264]:
import random

random.randint(20,45)

36

In [7]:
import time
import random
# include reviews, stars, description, cool, useful, funny
userlist = []
for idx, row in enumerate(restaurants.find()):
    num = idx + 1
    if num >= 376 and num <= 385:
        r = requests.get(row['url'], headers={'user-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(r.content, "html")
        pagination_div = soup.find("div", {"class": "pagination__373c0__1NjN5"})
        pages = int(pagination_div.find("span", {"class": "text__373c0__2pB8f"}).text.split("of")[-1].strip())
        for n in range(pages):
            st = n * 20
            updated_url = row['url'] + "&start="+ str(st)
            print(updated_url)
            updated_request = requests.get(updated_url, headers={'user-agent': 'Mozilla/5.0'})
            updated_soup = BeautifulSoup(updated_request.content, "html")
            users_div = updated_soup.findAll("div", {"class": "user-passport-info"})
            user_card_div = updated_soup.findAll("li", {"class": "lemon--li__373c0__1r9wz u-space-b3 u-padding-b3 border--bottom__373c0__uPbXS border-color--default__373c0__2oFDT"})
            stats_div = updated_soup.findAll("div", {"class": "user-passport-stats__373c0__2LjLz"})
            voter_div = updated_soup.findAll("div", {"class": "lemon--div__373c0__1mboc arrange__373c0__UHqhV vertical-align-middle__373c0__2TQsQ border-color--default__373c0__2oFDT"})
            for idx, user_div in enumerate(users_div):
                user = {}
                if user_div.find("a"):
                    href = user_div.find("a")['href']
                user_id = href.split("userid=")[-1]
                if user_div.find("a"):
                    url = "https://www.yelp.com" + user_div.find("a")['href']
                else:
                    url = ''
                user['userid'] = user_id
                user['restaurant_id'] = row['_id']
                user['name'] = user_div.findAll("span", {"class": "text__373c0__2pB8f"})[0].text
                user['url'] = url
                
                location = user_div.findAll("span", {"class": "text__373c0__2pB8f"})[-1].text.split(",")
                if len(location) == 1:
                    user['city'] = location[0].strip()
                else:
                    user['city'] = location[0].strip()
                    user['state'] = location[1].strip()
                for div in stats_div[idx].findAll("span", {"class": "text__373c0__2pB8f"}):
                    text = div.text
                    num = int(text.split(" ")[0])
                    key = text.split(" ")[-1]
                    if num == 1:
                        key = key + "s"
                    user[key] = text.split(" ")[0]
                user['stars'] = 0.0
                star_div = user_card_div[idx].select('div[class*="i-stars_"]')[0]
                if star_div:
                    user['stars'] = float(star_div['aria-label'].strip('star')[0])
                overall_div = user_card_div[idx].find("div", {"class": "arrange-unit-grid-column--8__373c0__2yTAx"})
                user['date'] = ''
                if overall_div and overall_div.find("div", {"class": "lemon--div__373c0__1mboc"}):
                    user['date'] = overall_div.find("div", {"class": "lemon--div__373c0__1mboc"}).findAll("span")[-1].text
                user['review'] = ''
                if overall_div.findAll("p"):
                    user['review'] = overall_div.findAll("p")[-1].text
                if user['review'] == 'Read more':
                    user['review'] = overall_div.findAll(attrs={"lang" : "en"})[0].text
                voter_cards = voter_div[idx].findAll("span", {"class": "text-size--small__373c0__3SGMi"})
                for v in voter_cards:
                    v_array = v.text.split(" ")
                    if len(v_array[0].strip()) != 0:
                        if len(v_array) == 2:
                            user[v_array[0]] = int(v_array[1])
                        else:
                            user[v_array[0]] = 0
                users.insert_one(user)
                userlist.append(user)
        processed.insert_one({'rowid': row['_id']})
#     break
userlist

https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=0
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=20
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=40
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=60
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=80
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=100
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=120
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=140
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=160
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=180
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=200
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&start=220
https://www.yelp.com/biz/kaisen-don-san-francisco-2?osq=Restaurants&st

https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1660
https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1680
https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1700
https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1720
https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1740
https://www.yelp.com/biz/magnolia-haight-san-francisco-3?osq=Restaurants&start=1760
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=0
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=20
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=40
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=60
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&s

https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1520
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1540
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1560
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1580
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1600
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1620
https://www.yelp.com/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco?osq=Restaurants&start=1640
https://www.yelp.com/biz/braised-bread-san-francisco-3?osq=Restaurants&start=0
https://www.yelp.com/biz/braised-bread-san-francisco-3?osq=Restaurants&start=20
https://www.yelp.com/biz/braised-bread-san-francisco-3?osq=Restaurants&start=40
https://www.yelp.com/biz/braised-bread-s

https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=240
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=260
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=280
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=300
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=320
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=340
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=360
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=380
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=400
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=420
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=440
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=460
https://www.yelp.com/biz/piqueos-san-francisco?osq=Restaurants&start=480
https://www.yelp.com/biz/piqueos-san-francisco?osq=

[{'userid': 'SgUv6nrd1uKtDvppvOmP-A',
  'restaurant_id': ObjectId('5e4f7cac3296483cf9246e85'),
  'name': 'Pauline W.',
  'url': 'https://www.yelp.com/user_details?userid=SgUv6nrd1uKtDvppvOmP-A',
  'city': 'Millbrae',
  'state': 'CA',
  'friends': '18',
  'reviews': '107',
  'photos': '1100',
  'stars': 5.0,
  'date': '1/3/2020',
  'review': 'My friend and I dropped by Kaisen DON after working out at the gym one evening. \xa0I actually had no idea that this place was so close to the gym and we were quite lucky to have found a place that sells great sushi for reasonable prices.The space is fairly small and we had made a reservation the day before so we didn\'t have to wait at all for our table. \xa0Here\'s what we ordered:1. Omega-3 Done (mine)2. Baked scallop roll (shared)3. Ingleside roll (shared)4. Rainbow Don (his)(these desserts were on the house!)5. Yuzu cheesecake (mine)6. Manjo mochi ice cream (his)Honestly, we almost couldn\'t finish! \xa0Leftover was a few more pieces of each r

In [109]:
# restaurant [1, 2]
userlist = []
for idx, row in enumerate(restaurants.find()):
    if idx == 2:
        r = requests.get('https://www.yelp.com/biz/the-house-san-francisco?osq=Restaurants&start=3840', headers={'user-agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(r.content, "html")
        users_div = updated_soup.findAll("div", {"class": "user-passport-info"})
        stats_div = updated_soup.findAll("div", {"class": "user-passport-stats__373c0__2LjLz"})
        
        for idx, user_div in enumerate(users_div):
            user = {}
            href = user_div.find("a")['href']
            user_id = href.split("userid=")[-1]
            url = "https://www.yelp.com" + user_div.find("a")['href']
            user['userid'] = user_id
            user['name'] = user_div.findAll("span", {"class": "text__373c0__2pB8f"})[0].text
            user['url'] = url
            location = user_div.findAll("span", {"class": "text__373c0__2pB8f"})[-1].text.split(",")
            print(location)
            if len(location) == 1:
                user['city'] = location[0].strip()
            else:
                user['city'] = location[0].strip()
                user['state'] = location[1].strip()
            for div in stats_div[idx].findAll("span", {"class": "text__373c0__2pB8f"}):
                text = div.text
                key = text.split(" ")[-1]
                user[key] = text.split(" ")[0]
#             users.insert_one(user)
            userlist.append(user)
#         processed.insert_one({'rowid': row['_id']})
#     break
userlist

['San Jose', ' CA']
['San Francisco', ' CA']
['Atlanta', ' GA']
['San Diego', ' CA']
['Waco', ' TX']
['Manhattan', ' NY']
['London', ' United Kingdom']
['Norwalk', ' CA']
['Fullerton', ' CA']
['West Islip', ' NY']
['Marina', ' CA']
['Walnut Creek', ' CA']
['Hong Kong']
['Ocean City', ' NJ']
['Shawnee On Delaware', ' PA']
['Bergenfield', ' NJ']
['San Antonio', ' TX']
['San Ramon', ' CA']
['Houston', ' TX']
['Las Vegas', ' NV']


[{'userid': 'on5J1IeBVlLBSG37OfqGTg',
  'name': 'Kathie T.',
  'url': 'https://www.yelp.com/user_details?userid=on5J1IeBVlLBSG37OfqGTg',
  'city': 'San Jose',
  'state': ' CA',
  'friends': '10',
  'reviews': '7',
  'photos': '11'},
 {'userid': 'cHi8XNYrOmP5u5W4IUwxEA',
  'name': 'Brian H.',
  'url': 'https://www.yelp.com/user_details?userid=cHi8XNYrOmP5u5W4IUwxEA',
  'city': 'San Francisco',
  'state': ' CA',
  'friends': '0',
  'reviews': '20',
  'photo': '1'},
 {'userid': 'aq7WTj6V86TdUNRTGNDLlw',
  'name': 'LaDonna R.',
  'url': 'https://www.yelp.com/user_details?userid=aq7WTj6V86TdUNRTGNDLlw',
  'city': 'Atlanta',
  'state': ' GA',
  'friends': '103',
  'reviews': '39',
  'photos': '55'},
 {'userid': 'P692ujWFLPsbnlsLSRgYrQ',
  'name': 'Wade T.',
  'url': 'https://www.yelp.com/user_details?userid=P692ujWFLPsbnlsLSRgYrQ',
  'city': 'San Diego',
  'state': ' CA',
  'friends': '12',
  'reviews': '76',
  'photos': '28'},
 {'userid': '07JBTQixFZ46YlKlPpHBow',
  'name': 'Tiffany R.',
  